# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [13]:
import subprocess

def extract_songs(srt_file, video_file, dry_run=False):
    with open(srt_file, 'r') as f:
        lines = f.readlines()
    
    song_start = None
    song_end = None
    songs = []

    for i in range(len(lines)):
        if i + 2 >= len(lines):
            break

        line = lines[i].strip()
        next_line = lines[i+1].strip()
        next_next_line = lines[i+2].strip()

        if '♪' in next_line:
            if song_start is None:
                song_start = line.split(" --> ")[0]
            song_end = line.split(" --> ")[1]

        if '♪' not in next_line and song_start is not None:
            songs.append((song_start, song_end))
            song_start = None
            song_end = None

    if dry_run:
        for song in songs:
            print("Song start: " + song[0] + ", Song end: " + song[1])
    else:
        for i, song in enumerate(songs):
            start = song[0]
            end = song[1]
            subprocess.run(["ffmpeg", "-i", video_file, "-ss", start, "-to", end,
                            "-c", "copy", f"output_{i}.mp4"])
extract_songs("../data/subtitles.srt", "../data/video.mp4", dry_run=True)

IndexError: list index out of range

In [207]:
import pysrt, re
def time_to_seconds(t):
    seconds = (t.hours * 60 + t.minutes) * 60 + t.seconds + t.milliseconds / 1000
    return seconds
def min_song(a,b):
    if a < b:
        return a
    else:
        return b
subs = pysrt.open('../data/subtitles.srt')
max_seconds = time_to_seconds(subs[-1].end)
def containsWords(text):
    # Use regex to check if text contains English chars
    return bool(re.search('[a-zA-Z]', text))
    
# Filter subs for those with "♪" in text
subs = [sub for sub in subs if ("♪" in sub.text or "♫" in sub.text) and containsWords(sub.text)]
threshold_padding = 170 # allow for 30 seconds of silence or short non-lyrical conversations within songs
start_minus = 5 # show 45 seconds prior
end_plus = 5 # show 5 seconds after
def st(start_time):
    return max(0, start_time-start_minus)
def et(end_time):
    return min(end_time+end_plus, max_seconds)

# Calculate the song times
song_times = []
start_time = time_to_seconds(subs[0].start)
end_time = time_to_seconds(subs[0].end)
for i in range(len(subs)-1):
    if subs[i].index == 1167:
        print('here', subs[i].text)
    # Get the start and end time of the current and next sub
    start_current = time_to_seconds(subs[i].start)
    end_current = time_to_seconds(subs[i].end)
    start_next = time_to_seconds(subs[i+1].start)
    end_next = time_to_seconds(subs[i+1].end)
    # Determine if the two subs are part of the same song
    if (start_next - end_current) > threshold_padding:
        # If not, add the current song times to the list and reset the start and end times
        song_times.append([ms(start_time),et(end_time)])
        start_time = start_next
        end_time = end_next
    else:
        # If so, extend the end time of the current song
        end_time = min_song(end_next,end_current)
# Add the last song time
song_times.append([ms(start_time),et(end_time)])

# Print the song times
cum_duration = 0
print(len(song_times))
for song_time in song_times:
    print(f"Song Time: {song_time[0]} - {song_time[1]}")
    cum_duration += song_time[1] - song_time[0]
print(cum_duration/60, 'minutes')

here - <i>♪ Gonna drown them out ♪</i>
- <i>♪ Oh ♪</i>
9
Song Time: 6.073 - 667.458
Song Time: 934.601 - 974.696
Song Time: 1520.653 - 1819.007
Song Time: 2211.677 - 2440.661
Song Time: 2960.728 - 3145.402
Song Time: 3386.954 - 3611.3
Song Time: 3814.38 - 4433.222
Song Time: 4709.578 - 4740.329
Song Time: 5022.521 - 6214.835
57.99574999999998 minutes


In [193]:
subs[4].index

7

In [179]:
%cd ../data

/Users/itai/dev/musical-trimmer/data


In [208]:
import subprocess
input_video, input_subtitles, output_video = 'video.mp4', 'subtitles.srt', 'output_shorter.mkv'
temp_mkv = 'temp.mkv'
subprocess.run(["ffmpeg", "-i", input_video, "-i", input_subtitles, "-c", "copy", "-c:s", "srt", "-metadata:s:s:1", "language=eng", "-map", "0", "-map", "1", "-map_metadata", "0", temp_mkv, "-y"])
with open('../data/song_times.txt', 'w') as f:
    for song_time in song_times:
        f.write(f"file {temp_mkv}\n")
        f.write(f"inpoint {song_time[0]}\n")
        f.write(f"outpoint {song_time[1]}\n")
subprocess.run(["ffmpeg", "-f", "concat", "-safe", "0", "-i", "../data/song_times.txt", "-c", "copy", output_video, "-y"])
subprocess.run(["rm", temp_mkv])

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

CompletedProcess(args=['rm', 'temp.mkv'], returncode=0)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()